In [4]:
import json
import requests

### 도로명주소로 좌표 얻기

In [287]:
url = 'https://dapi.kakao.com/v2/local/search/address.json?query=서울 마포구 모래내로1길 20'
headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} 
result = json.loads(str(requests.get(url, headers=headers).text))
# result

In [288]:
def getLatLng(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + addr
    headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']), float(match_first['x'])
getLatLng('서울 마포구 모래내로1길 20')

(37.5656948713242, 126.903284318892)

---

## 좌표로 행정동 / 법정동 얻기

In [289]:
import requests
import json
import pandas as pd

In [290]:
url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.1086228&y=37.4012191" # 여기 x, y값에 위경도 대입하면 될듯 

headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} # 이건 내 api 발급받은거 

In [291]:
result = json.loads(str(requests.get(url, headers=headers).text))
result['documents'][0] 
# B : 법정동 의미

{'region_type': 'B',
 'code': '4113510900',
 'address_name': '경기도 성남시 분당구 삼평동',
 'region_1depth_name': '경기도',
 'region_2depth_name': '성남시 분당구',
 'region_3depth_name': '삼평동',
 'region_4depth_name': '',
 'x': 127.1163593869371,
 'y': 37.40612091848614}

In [292]:
result['documents'][0]['region_3depth_name']

'삼평동'

In [293]:
def getDong(long, lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(str(long), str(lat)) 
    # 경도 / 위도 formatting 
    headers = {"Authorization": "KakaoAK a12d3a71c08429eed2789c17a0c354ac"} # 내 인증키 
    result = json.loads(str(requests.get(url, headers=headers).text))
    dong = result['documents'][0]['region_3depth_name']
    return dong

In [294]:
getDong(127.11786562005, 37.528682790758)  # 풍납동 현대아파트 위도 경도 

'풍납동'

In [61]:
charger = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/api_charger.txt')
print(charger.shape)
charger.head()

(11673, 7)


,statNm,statId,addr,lat,lng,bnm,busiNm
0,종묘 공영주차장,ME000001,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,환경부
1,세종로 공영주차장,ME000002,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,환경부
2,그랜드앰배서더 서울,ME000003,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,환경부
3,한강진역 공영주차장,ME000004,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,환경부
4,기아차 성동서비스센터,ME000005,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,환경부


In [45]:
longlat = charger[['lng', 'lat']]
print(len(longlat))
longlat.head()

11673


,lng,lat
0,126.995880,37.571076
1,126.976011,37.573611
2,127.002350,37.559352
3,127.002804,37.540085
4,127.064226,37.544834


In [47]:
### 시간 매우 오래 걸림 ### 

dong = []

for i in longlat.index: 
    dong.append(getDong(longlat['lng'][i], longlat['lat'][i]))
    
longlat['법정동'] = dong
longlat.head()

<ipython-input-47-8af8050ecced>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longlat['법정동'] = dong


,lng,lat,2,법정동
0,126.995880,37.571076,0,훈정동
1,126.976011,37.573611,1,세종로
2,127.002350,37.559352,2,장충동2가
3,127.002804,37.540085,3,한남동
4,127.064226,37.544834,4,성수동2가


In [53]:
# longlat.drop('2', axis=1, inplace=True)

In [54]:
print(longlat.shape)
longlat.head(n=10)

(11673, 3)


,lng,lat,법정동
0,126.995880,37.571076,훈정동
1,126.976011,37.573611,세종로
2,127.002350,37.559352,장충동2가
3,127.002804,37.540085,한남동
4,127.064226,37.544834,성수동2가
5,127.045526,37.566094,마장동
6,127.047938,37.580650,전농동
7,127.061848,37.598454,이문동
8,127.081540,37.580612,면목동
9,127.040024,37.602083,하월곡동


In [55]:
# longlat.to_csv('/Users/kimmh/Desktop/서데캠/data/위경도_법정동.txt', sep=',', index=False)

### 전기차 충전소 현황 데이터에 법정동 열 추가 (위도/경도 기준) 

In [247]:
charger = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/api_charger.txt')
print(charger.shape)
charger.head()

(11673, 7)


,statNm,statId,addr,lat,lng,bnm,busiNm
0,종묘 공영주차장,ME000001,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,환경부
1,세종로 공영주차장,ME000002,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,환경부
2,그랜드앰배서더 서울,ME000003,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,환경부
3,한강진역 공영주차장,ME000004,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,환경부
4,기아차 성동서비스센터,ME000005,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,환경부


In [248]:
charger['법정동'] = longlat['법정동']
charger.head()

,statNm,statId,addr,lat,lng,bnm,busiNm,법정동
0,종묘 공영주차장,ME000001,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,환경부,훈정동
1,세종로 공영주차장,ME000002,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,환경부,세종로
2,그랜드앰배서더 서울,ME000003,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,환경부,장충동2가
3,한강진역 공영주차장,ME000004,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,환경부,한남동
4,기아차 성동서비스센터,ME000005,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,환경부,성수동2가


In [249]:
charger.drop(['statNm', 'statId'], axis=1, inplace=True) 
print(charger.shape)
charger.head()

(11673, 6)


,addr,lat,lng,bnm,busiNm,법정동
0,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,환경부,훈정동
1,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,환경부,세종로
2,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,환경부,장충동2가
3,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,환경부,한남동
4,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,환경부,성수동2가


In [250]:
from itertools import repeat
charger['cnt'] = list(repeat(1, len(charger)))

print(charger.shape)
charger.head()

(11673, 7)


,addr,lat,lng,bnm,busiNm,법정동,cnt
0,서울특별시 종로구 종로 157,37.571076,126.995880,환경부,환경부,훈정동,1
1,서울특별시 종로구 세종대로 189,37.573611,126.976011,환경부,환경부,세종로,1
2,서울특별시 중구 동호로 287,37.559352,127.002350,환경부,환경부,장충동2가,1
3,서울특별시 용산구 한남동 산10-84,37.540085,127.002804,환경부,환경부,한남동,1
4,서울특별시 성동구 성수이로26길 61,37.544834,127.064226,환경부,환경부,성수동2가,1


In [254]:
charger[charger['addr']==' 입구좌측, 지하 각 1기']

,addr,lat,lng,bnm,busiNm,법정동,cnt
57,"입구좌측, 지하 각 1기",37.488301,126.934612,환경부,환경부,봉천동,1


In [255]:
charger['addr'] = charger['addr'].replace(' 입구좌측, 지하 각 1기', '서울특별시 관악구 봉천로21길')

In [257]:
dt = charger.groupby(['addr', 'lat', 'lng', '법정동'])
gr = dt.sum('cnt')
gr = gr.reset_index()

print(gr.shape)
print(gr['addr'].nunique())
gr.head()

(3548, 5)
3036


,addr,lat,lng,법정동,cnt
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2


### lat=33.0 / lng=124.0 : 위도 경도가 결측치인 값들 > 법정동 비어있음 ' ' (19개) 

In [258]:
gr[gr['법정동']=='']

,addr,lat,lng,법정동,cnt
791,서울특별시 광진구 능동로 19,33.0,124.0,,4
884,서울특별시 구로구 경인로8길 64,33.0,124.0,,10
956,서울특별시 구로구 디지털로34길 27,33.0,124.0,,2
1190,서울특별시 노원구 동일로 1127,33.0,124.0,,3
1310,서울특별시 도봉구 도봉로136가길 73,33.0,124.0,,8
1314,서울특별시 도봉구 도봉로169길 31,33.0,124.0,,3
1381,서울특별시 도봉구 해등로16길 12,33.0,124.0,,5
1402,서울특별시 동대문구 망우로 95,33.0,124.0,,8
1497,서울특별시 동대문구 황물로15길 25,33.0,124.0,,6
1581,서울특별시 동작구 서달로 91,33.0,124.0,,10


In [259]:
gr.iloc[791] = gr.iloc[791].replace('', '자양동')
gr.iloc[884] = gr.iloc[884].replace('', '오류동')
gr.iloc[956] = gr.iloc[956].replace('', '구로동')
gr.iloc[1190] = gr.iloc[1190].replace('', '공릉동')
gr.iloc[1310] = gr.iloc[1310].replace('', '창동')
gr.iloc[1314] = gr.iloc[1314].replace('', '도봉동')
gr.iloc[1381] = gr.iloc[1381].replace('', '창동')
gr.iloc[1402] = gr.iloc[1402].replace('', '휘경동')
gr.iloc[1497] = gr.iloc[1497].replace('', '답십리동')
gr.iloc[1581] = gr.iloc[1581].replace('', '흑석동')
gr.iloc[1684] = gr.iloc[1684].replace('', '신공덕동')
gr.iloc[2333] = gr.iloc[2333].replace('', '안암동1가')
gr.iloc[2438] = gr.iloc[2438].replace('', '장위동')
gr.iloc[2672] = gr.iloc[2672].replace('', '풍납동')
gr.iloc[2855] = gr.iloc[2855].replace('', '대림동')
gr.iloc[2888] = gr.iloc[2888].replace('', '대림동')
gr.iloc[3228] = gr.iloc[3228].replace('', '녹번동')
gr.iloc[3417] = gr.iloc[3417].replace('', '신당동')
gr.iloc[3494] = gr.iloc[3494].replace('', '상봉동')

In [260]:
gr[gr['법정동']=='']  ### 결측치 잘 채워졌음을 확인 

,addr,lat,lng,법정동,cnt


In [261]:
gr[gr['addr']=='서울특별시 서초구 신반포로 270']  # 반포자이 - 같은 주소인데 충전소가 여러개로 입력된 경우

,addr,lat,lng,법정동,cnt
2066,서울특별시 서초구 신반포로 270,37.504331,127.017304,반포동,3
2067,서울특별시 서초구 신반포로 270,37.504800,127.016456,반포동,3
2068,서울특별시 서초구 신반포로 270,37.505116,127.015630,반포동,3
2069,서울특별시 서초구 신반포로 270,37.505215,127.016806,반포동,1
2070,서울특별시 서초구 신반포로 270,37.505449,127.016727,반포동,3
2071,서울특별시 서초구 신반포로 270,37.505629,127.015675,반포동,3
2072,서울특별시 서초구 신반포로 270,37.506062,127.016445,반포동,3
2073,서울특별시 서초구 신반포로 270,37.506287,127.010710,반포동,3
2074,서울특별시 서초구 신반포로 270,37.506436,127.010914,반포동,1
2075,서울특별시 서초구 신반포로 270,37.506828,127.011683,반포동,3


#### 위도/경도 조금씩 다름 > 차후에 지도에 점 찍을 경우 대비하여 합치지 않는 걸로

#### 법정동 코드 추가 

In [262]:
match = pd.read_csv('/Users/kimmh/Desktop/서데캠/data/행정_법정동코드_매칭.txt')
match = match.rename(columns={'동리명':'법정동'})
match.drop(['행정동코드', '읍면동명', 'match'], axis=1, inplace=True)
match.reset_index()


print(match.shape)
match.head()

(742, 2)


,법정동코드,법정동
0,1111010100,청운동
1,1111010200,신교동
2,1111010300,궁정동
3,1111010400,효자동
4,1111010500,창성동


In [263]:
merged = pd.merge(gr, match, how='left', on='법정동')
print(merged.shape)
merged.head()

(10794, 6)


,addr,lat,lng,법정동,cnt,법정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1.168010e+09
1,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1.168010e+09
2,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1.168011e+09
3,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1.168011e+09
4,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1.168010e+09


In [269]:
data = merged.drop_duplicates(['addr','lat', 'lng'], keep='first', ignore_index=True)
print(data.shape)
data.head()
# data['법정동코드'] = data['법정동코드'].astype(int)

(3548, 6)


,addr,lat,lng,법정동,cnt,법정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1.168010e+09
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1.168011e+09
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1.168010e+09
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1.168010e+09
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1.168011e+09


In [270]:
data.isna().sum()

addr      0
lat       0
lng       0
법정동       0
cnt       0
법정동코드    11
dtype: int64

#### 위/경도 > 법정동 변환 코드에서 오류가 난 경우인 듯

In [283]:
na = data[data['법정동코드'].isna()]
na

,addr,lat,lng,법정동,cnt,법정동코드
655,서울특별시 강서구 허준로 47,37.500000,126.800000,춘의동,3,NaN
849,서울특별시 구로구 가마산로25길 33(구로동),37.703795,126.903946,고양동,1,NaN
1061,서울특별시 금천구 디지털로9길 33,35.122107,126.907958,봉선동,3,NaN
1229,서울특별시 노원구 상계로 267,37.405561,127.423366,산북면,2,NaN
1602,"서울특별시 동작구 여의대방로10길 97 (신대방동, 남부수도사업소)",37.429300,126.915400,석수동,4,NaN
2266,"서울특별시 성동구 무학로12길 11 (홍익동, 청계대주파크빌)",37.340200,127.189000,모현읍,1,NaN
2439,서울특별시 성북구 장월로 160,38.025677,127.072628,전곡읍,6,NaN
2935,서울특별시 영등포구 여의동로 97,35.693199,128.456344,현풍읍,2,NaN
3129,서울특별시 은평구 불광로2길 33,37.612951,127.937666,영귀미면,10,NaN
3274,서울특별시 종로구 율곡로 179,35.611307,126.521692,진서면,2,NaN


### gr에서 먼저 법정동 수정 필요해보임

#### 위 12개 다 검색해보고 수정하는 걸로

In [281]:
import warnings
warnings.filterwarnings(action='ignore')

In [284]:
data.iloc[61] = data.iloc[61].replace('금촌동', '도곡동').fillna(168011800)
data.iloc[655] = data.iloc[655].replace('춘의동', '가양동').fillna(1150010400)
data.iloc[849] = data.iloc[849].replace('고양동', '구로동').fillna(1153010200)
data.iloc[1061] = data.iloc[1061].replace('봉선동', '가산동').fillna(1154510100)
data.iloc[1229] = data.iloc[1229].replace('산북면', '상계동').fillna(1135010500)
data.iloc[1602] = data.iloc[1602].replace('석수동', '신대방동').fillna(1159010900)
data.iloc[2266] = data.iloc[2266].replace('모현읍', '홍익동').fillna(1120010300)
data.iloc[2439] = data.iloc[2439].replace('전곡읍', '장위동').fillna(1129013800)
data.iloc[2935] = data.iloc[2935].replace('현풍읍', '여의도동').fillna(1156011000)
data.iloc[3129] = data.iloc[3129].replace('영귀미면', '불광동').fillna(1138010300)
data.iloc[3274] = data.iloc[3274].replace('진서면', '원남동').fillna(1111015900)

In [286]:
data.isna().sum()

addr     0
lat      0
lng      0
법정동      0
cnt      0
법정동코드    0
dtype: int64

## 모든 결측치 처리 완료! 

In [296]:
data['법정동코드'] = data['법정동코드'].astype(int)
data.head(n=10)

,addr,lat,lng,법정동,cnt,법정동코드
0,서울특별시 강남구 테헤란로 305(역삼동),37.503534,127.043759,역삼동,1,1168010100
1,서울특별시 강남구 강남대로 128길 20 영동프라자,37.509900,127.023600,논현동,1,1168010800
2,서울특별시 강남구 강남대로 298 (역삼동),37.490051,127.032163,역삼동,2,1168010100
3,서울특별시 강남구 강남대로 362,37.495268,127.029404,역삼동,4,1168010100
4,서울특별시 강남구 강남대로120길 33,37.508248,127.026766,논현동,2,1168010800
5,서울특별시 강남구 강남대로146길 28,37.515043,127.022903,논현동,5,1168010800
6,서울특별시 강남구 강남대로84길 33,37.497549,127.031082,역삼동,2,1168010100
7,서울특별시 강남구 개포동 13-2,37.494910,127.079280,개포동,1,1168010300
8,서울특별시 강남구 개포로,37.494966,127.081799,일원동,2,1168011400
9,서울특별시 강남구 개포로 25길 32,37.481510,127.048510,개포동,1,1168010300


### 최종 데이터 저장 

In [297]:
data.to_csv('/Users/kimmh/Desktop/서데캠/data/api_charger_processed.txt', sep=',', index=False)